# Actividad Integradora 3.4 Resaltador de sintaxis

Fernando Daniel Monroy Sánchez - A01750536

Eugenio Andrés Mejía Fanjón - A01412143


El siguiente trabajo es un resaltador de sintaxis del lenguaje de programación Python, el cuál es un lenguaje interpretado con el objetivo de habilitar de manera rápida y sencilla múltiples funcionalidades a través de librerías poderosas y una sintáxis fácil de entender.


## Introducción a gramática de Python


Python deja al desarrollador únicamente la tarea de definir aquello que se quiere realizar en un mayor nivel, y el programa y sus múltiples librerías disponibles se encargan de dar forma a todas las instrucciones en un bajo nivel usando estándares de la industria y configuraciones predeterminadas. Debido a estas razones, la sintaxis es sencilla y limpia.

Un programa de python se compone de múltiples tokens a los que se les hace _parsing_. Existen diferentes tipos de tokens en el programa, y cada uno de estos recibe únicamente un subconjunto del lenguaje, dado por caracteres aceptados por expresiones regulares.

Debido a la gran cantidad de tokens de la [gramática completa](https://docs.python.org/3/reference/grammar.html), que incluye muchos casos específicos y poco comunes, en el presente trabajo se abordarán unicamente los más comunes y usados el 99% de las veces.


### Tokens "discretos"


| Token                                                                                              | Ejemplos                                     |
| -------------------------------------------------------------------------------------------------- | -------------------------------------------- |
| [Operadores](https://docs.python.org/3/reference/lexical_analysis.html#operators)                  | `+`, `-`, `*`, `**`, $\ldots$                |
| [Delimitadores](https://docs.python.org/3/reference/lexical_analysis.html#delimiters)              | `(`, `)`, `[`, `]`, $\ldots$                 |
| [Palabras clave](https://docs.python.org/3/reference/lexical_analysis.html#keywords)               | `False`, `await`, `else`, `import`, $\ldots$ |
| [Palabras clave suaves](https://docs.python.org/3/reference/lexical_analysis.html#soft-keywords)   | `match`, `case`, `type`, `_`                 |
| [Secuencias de escape](https://docs.python.org/3/reference/lexical_analysis.html#escape-sequences) | `\'`, `\"`, `\\`, $\ldots$                   |


### Tokens "continuos"


| Token                                                                                                  | Ejemplos                                                       |
| ------------------------------------------------------------------------------------------------------ | -------------------------------------------------------------- |
| [Strings y bytes](https://docs.python.org/3/reference/lexical_analysis.html#string-and-bytes-literals) | `'hola'`, `"mundo"`, `f"f-string"`, `'\nEscape'`, $\ldots$     |
| [Números enteros](https://docs.python.org/3/reference/lexical_analysis.html#integer-literals)          | `3`, `750336`, `0o377`, `0xdeadbeef`, `0b_1110_0101`, $\ldots$ |
| [Números flotantes](https://docs.python.org/3/reference/lexical_analysis.html#floating-point-literals) | `3.14`, `10.`, `.001`, `1e100`, $\ldots$                       |
| [Números imaginarios](https://docs.python.org/3/reference/lexical_analysis.html#imaginary-literals)    | `3.14j`, `10.j`, `10j`, $\ldots$                               |
| [Comentarios](https://docs.python.org/3/reference/lexical_analysis.html#comments)                      | `#Hola`, `####`, $\ldots$                                      |
| Variable                                                                                               | `var`, `_12`, `match_`, $\ldots$                               |


## Expresiones regulares de parsing


Importar librerías necesarias


In [1]:
# Para expresiones regulares
import re
# Para codificar en HTML el archivo de salida
import html

Establecer reglas de parsing de cada token discreto


In [2]:
discrete_patterns = {
    "operator": re.compile(r'\*\*|//|<=|>=|==|!=|:=|\+|-|\*|/|%|@|<<|>>|&|\||\^|~|<|>'),

    "delimiters": re.compile(r'\(|\)|\[|\]|\{|\}|\,|\:|\.|\;|\@|\=|\->|\+=|-=|\*=|/=|//=|%=|@=|&=|\|=|\^=|>>=|<<=|\*\*='),

    "keyword": re.compile(r'(?<!\w)(False|await|else|import|pass|None|break|except|in|raise|True|class|finally|is|return|and|continue|for|lambda|try|as|def|from|nonlocal|while|assert|del|global|not|with|async|elif|if|or|yield)(?!\w)'),

    "soft_keyword": re.compile(r'\s+(match|case|type|_)\s+'),

    "escape_sequences": re.compile(r'\\(?:\n|\\|\'|\"|a|b|f|n|r|t|v)')
}

Establecer reglas de parsing de cada token continuo


In [3]:
cont_patterns = {

    "string": re.compile(r"""
        (?:[rRuUfF]*)           # Empezar el string con un prefijo opcional
        (?:
            '[^'\\]*(?:\\.[^'\\]*)*'    # Cuando se define usando '
            | "[^"\\]*(?:\\.[^"\\]*)*"  # Cuando se define usando "
            | '''.*?'''                 # Cuando se define usando '''
        )
    """, re.DOTALL | re.VERBOSE),


    "int": re.compile(r'''
    # -------------------------------------------------------
    (
            [1-9][_\d]*         # Número base 10, que sea > 0
        |   0[_0]*              # Número base 10, que sea == 0
        |   0[bB][_01]+         # Número base 2 (binario)
        |   0[oO][_0-7]+        # Número base 8 (oct)
        |   0[xX][_\da-fA-F]+   # Número base 16 (hex)
    )
    # -------------------------------------------------------
    ''', re.VERBOSE),


    "float": re.compile(r'''
    # -------------------------------------------------------
    (
        \d(?:[_\d]*)                  # Siempre empezar con al menos 1 dígito
        (?:
                \.\d(?:[_\d]*)?             # Punto decimal y después 1 o más dígitos
            |   \.                          # Punto decimal y después 0 dígitos
        )
        (?:
            [eE][+-]?\d(?:[_\d]*)           # Opcionalmente, agregar exponente positivo o negativo
        )?
    )
    # -------------------------------------------------------
    ''', re.VERBOSE),


    "comment": re.compile(r"(#.*$)"),

    "variable": re.compile(r"[a-zA-Z_]\w*"),

    "method": re.compile(r"(?<=\.)([a-zA-Z_]\w*)(?=\()")
}

Join both pattern types into a single dictionary


In [4]:
cont_patterns.update(discrete_patterns)
patterns = cont_patterns

print(patterns.keys())

dict_keys(['string', 'int', 'float', 'comment', 'variable', 'method', 'operator', 'delimiters', 'keyword', 'soft_keyword', 'escape_sequences'])


### Notas

Recordar que, de cierta manera, el token que debe siempre tener la mayor prioridad es el comment, y luego el string, pues ambos pueden "contener" matches de otros tokens sin que necesariamente sean ciertos, e.g.:

```python
# variable = 'False'
variable = 'False'
variable = False
```

Otro punto importante es que, cuando el token no sea string, si se define una variable usando un keyword reservado se debe de mostrar la sintaxis como si la variable fuera la keyword. Esto es cierto pues en teoría no se debería de nombrar ninguna variable con un keyword. E.g.:

```python
class = 'hola'
break = None
```

La prioridad u orden de ejecución de cada pattern a encontrar en el texto por tanto debería quedar (de manera preliminar) como:

$\text{Comment} \rightarrow \text{String} \rightarrow \text{Keyword} \rightarrow \text{Method} \rightarrow \text{Variable} \rightarrow \text{Float} \rightarrow \text{All}$


Considerando este orden de ejecución, reordenar el diccionario


In [5]:
high_priority = ["comment", "string", "keyword", "method", "variable", "soft_keyword", "float"]

order = high_priority + [k for k in patterns.keys() if k not in high_priority]

patterns = {k : patterns[k] for k in order}

print(patterns.keys())

dict_keys(['comment', 'string', 'keyword', 'method', 'variable', 'soft_keyword', 'float', 'int', 'operator', 'delimiters', 'escape_sequences'])


Función para realizar parsing de archivo proporcionado y guardar HTML


In [6]:
def parse(file_name):

    with open(file_name, 'r') as file:
        # Every line of the code in which we will replace matches
        code_to_match = [line.rstrip() for line in file]
        # Copy of the code in which we will save the parsed code in HTML
        code_to_parse = [line for line in code_to_match]

    # List of tuples to save every tag found and its index
    code_tokens = []

    # Iterate through every line in the code
    for i in range(len(code_to_match)):
        line_tokens = []
        line_to_parse = code_to_parse[i]
        # Iterate through every pattern to match
        for token, pattern in patterns.items():
            # For every non-overlapping match
            matches = pattern.finditer(line_to_parse)
            indices = [m.span() for m in matches]
            for start_idx, end_idx in indices:
                # If the line_to_match gets completely emptied out, break loop
                if len(line_to_parse.strip()) == 0: break
                # Save element and its index into the code_tokens list
                line_tokens.append((line_to_parse[start_idx:end_idx], token, start_idx, end_idx))
                # Remove the match from the line_to_parse
                line_to_parse = line_to_parse[:start_idx] + ' '*(end_idx-start_idx) + line_to_parse[end_idx:]

            code_to_parse[i] = line_to_parse

        code_tokens.append(line_tokens)

    # Order every line by each element's starting index
    for i in range(len(code_tokens)):
        code_tokens[i].sort(key = lambda line_tokens: line_tokens[2])

    # Convert every found element to an HTML tag in order
    for i in range(len(code_tokens)):
        # print(code_to_match[i])
        code_to_parse[i] = '<p>' + code_to_parse[i]
        offset = 3 # Starting offset of 3 because of starting <p>
        for content, token, start_idx, end_idx in code_tokens[i]:
            encoded_content = html.escape(content)
            content_tag = f'<span class="{token}">{encoded_content}</span>'
            content_tag_len = len(content_tag)
            code_to_parse[i] = code_to_parse[i][:offset+start_idx] + content_tag + code_to_parse[i][offset+end_idx:]
            offset += content_tag_len - len(content)
        code_to_parse[i] += '</p>'
        # print(code_to_parse[i])

    # Convert list of lines to a single string
    parsed_code = "\n".join(code_to_parse)

    # With the final string, convert all the code's original whitespaces (such as indentation, spacing, etc.) back to displayable, HTML non-breaking spaces (nbsp's)
    spaces_matches = re.finditer(r'(?<=<\/span>)(\s+)(?=<span.*?>)|(?<=<p>)(\s+)(?=<span)', parsed_code)
    spaces_indices = [m.span() for m in spaces_matches]
    offset = 0
    new_content = '&nbsp;'
    for start_idx, end_idx in spaces_indices:
        for i in range(start_idx, end_idx):
            parsed_code = parsed_code[:offset+start_idx] + new_content + parsed_code[offset+start_idx:]
            offset += len(new_content)

    return parsed_code


parsed_code = parse('example.py')
print(f"Preview:\n{parsed_code[:1000]}")

Preview:
<p><span class="comment"># Variables and Data Types</span></p>
<p><span class="variable">name</span>&nbsp; <span class="delimiters">=</span>&nbsp; <span class="string">&quot;Alice&quot;</span></p>
<p><span class="variable">age</span>&nbsp; <span class="delimiters">=</span>&nbsp; <span class="int">30</span></p>
<p><span class="variable">height</span>&nbsp; <span class="delimiters">=</span>&nbsp; <span class="float">5.6</span></p>
<p><span class="variable">is_student</span>&nbsp; <span class="delimiters">=</span>&nbsp; <span class="keyword">True</span></p>
<p></p>
<p><span class="comment"># Basic Operations</span></p>
<p><span class="variable">result</span>&nbsp; <span class="delimiters">=</span>&nbsp; <span class="int">10</span>&nbsp; <span class="operator">+</span>&nbsp; <span class="int">5</span></p>
<p><span class="variable">difference</span>&nbsp; <span class="delimiters">=</span>&nbsp; <span class="int">20</span>&nbsp; <span class="operator">-</span>&nbsp; <span class="int

Improve the syntax of the HTML file with BeautifulSoup and then write it out to an `index.html` file


In [7]:
content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Python Script</title>
    <link rel="stylesheet" href="styles.css">
</head>
<body>
""" + parsed_code + """
</body>
</html>
"""

output_file_name = 'index.html'
with open(output_file_name, 'w') as file:
    file.write(content)

## Referencias


Python Software Foundation (2024). _Python Language Reference_. https://docs.python.org/3/reference/index.html

Python Software Foundation (2024). _Full Grammar Specification_. https://docs.python.org/3/reference/grammar.html
